Load all accessible cyceling data from tfl.gov.uk S3 buckets3s3

In [1]:
import boto3

from botocore import UNSIGNED
from botocore.config import Config
from botocore.handlers import disable_signing
from pathlib import Path

#s3 = boto3.client('s3',config=Config(signature_version=UNSIGNED))
s3 = boto3.resource('s3')
s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)

tfl_bucket = s3.Bucket('cycling.data.tfl.gov.uk')

In [7]:
print(tfl_bucket.get_available_subresources())

['Acl', 'Cors', 'Lifecycle', 'LifecycleConfiguration', 'Logging', 'Notification', 'Object', 'Policy', 'RequestPayment', 'Tagging', 'Versioning', 'Website']


In [ ]:
from pathlib import Path


for res in tfl_bucket.objects.filter(Prefix='usage-stats/'):
    print(res.key,res.key.split('/')[-1])
    try:
        path = Path('data') / Path(res.key.split('/')[-1])
        print("Save to",path)
        tfl_bucket.download_file(res.key,str(path))
    except Exception as e:
        print(e)
        continue

In [2]:
import pandas as pd
import dask.dataframe as ddf
from tqdm.dask import TqdmCallback
from tqdm import tqdm
from pathlib import Path

In [13]:
files = [ x for x in Path('data').iterdir() if x.suffix == '.csv']

In [14]:
for file in tqdm(files):
    df = pd.read_csv(file)
    df.to_parquet(Path('data')/Path('compressed')/Path(file.stem+'.parquet'))


100%|██████████| 363/363 [08:46<00:00,  1.45s/it]


In [3]:
parquetta = [ x for x in (Path('data')/'compressed').iterdir() ]

In [4]:
df_list = list()
for file in tqdm(parquetta):
    df_list.append(pd.read_parquet(file))
gdf = pd.concat(df_list)
print(not gdf['Rental Id'].is_unique)

100%|██████████| 363/363 [01:11<00:00,  5.06it/s]


True


In [5]:
print(gdf.size)

1626367100


In [ ]:
gdf['Start Date'] = pd.to_datetime(gdf['Start Date'],exact=False,infer_datetime_format=True)

In [9]:
dgdf = ddf.from_pandas(gdf,32)

MemoryError: 

In [8]:
with TqdmCallback(desc="compute"):
    dgdf['End Date'] = ddf.to_datetime(dgdf['End Date'],exact=False,error='coerce',infer_datetime_format=True)
    dgdf['Start Date'] = ddf.to_datetime(dgdf['Start Date'],exact=False,error='coerce',infer_datetime_format=True)

KeyboardInterrupt: 